# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Bañados  ->  E. Bañados  |  ['E. Bañados']
F. Walter  ->  F. Walter  |  ['F. Walter']
H. Baehr  ->  H. Baehr  |  ['H. Baehr']
Arxiv has 38 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2505.00080


extracting tarball to tmp_2505.00080...

 done.


F. Walter  ->  F. Walter  |  ['F. Walter']


Found 54 bibliographic references in tmp_2505.00080/main.bbl.
Error retrieving bib data for farina_mapping_2017: 'author'
Retrieving document from  https://arxiv.org/e-print/2505.00363


extracting tarball to tmp_2505.00363...

 done.


Found 67 bibliographic references in tmp_2505.00363/shearing_particles.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.00080-b31b1b.svg)](https://arxiv.org/abs/2505.00080) | **Quasar lifetime measurements from extended Ly$α$ nebulae at $z\sim 6$**  |
|| D. Ďurovčíková, et al. -- incl., <mark>E. Bañados</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2025-05-02*|
|*Comments*| *14 pages, 7 figures*|
|**Abstract**|            The existence of billion-solar-mass black holes hosted in luminous quasars within the first gigayear of cosmic history poses a challenge to our understanding of supermassive black hole (SMBH) growth. The problem is further exacerbated by the very short quasar lifetimes of $t_{\rm Q}\lesssim 10^6$ years, as derived from the extent of their proximity zone (PZ) sizes observed in the quasars' rest-UV spectra. However, the quasar lifetime estimates based on the extents of the proximity zones may be underestimated, as time-variable obscuration effects might have limited the quasars' emission along our sightline in the past. In this work, we present independent quasar lifetime measurements for six quasars at $z \sim 6$ leveraging the extended nebular emission perpendicular to our line-of-sight. We use observations from the Very Large Telescope/Multi-Unit Spectroscopic Explorer (MUSE) to search for extended Ly$\alpha$ emission in the circumgalactic medium around quasars with small proximity zones and estimate their lifetimes as the light travel time between the SMBH and the outer edge of the nebula. We find agreement between the independent lifetime estimates. For one object we find a proximate absorption system prematurely truncating the extent of the quasar's proximity zone, which thus results in an expected discrepancy between the lifetime estimates. Our results provide further evidence that the quasars' current accretion episode has only recently begun, challenging our models of SMBH growth.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.00363-b31b1b.svg)](https://arxiv.org/abs/2505.00363) | **Dust density enhancements and the direct formation of planetary cores in gravitationally unstable discs**  |
|| K. Rice, et al. -- incl., <mark>H. Baehr</mark> |
|*Appeared on*| *2025-05-02*|
|*Comments*| *15 pages, 11 figures, accepted for publication in Monthly Notices of the Royal Astronomical Society*|
|**Abstract**|            Planet formation via core accretion involves the growth of solids that can accumulate to form planetary cores. There are a number of barriers to the collisional growth of solids in protostellar discs, one of which is the drift, or metre, barrier. Solid particles experience a drag force that will tend to cause them to drift towards the central star in smooth, laminar discs, potentially removing particles before they grow large enough to decouple from the disc gas. Here we present 3-dimensional, shearing box simulations that explore the dynamical evolution of solids in a protostellar disc that is massive enough for the gravitational instability to manifest as spiral density waves. We expand on earlier work by considering a range of particle sizes and find that the spirals can still enhance the local solid density by more than an order of magnitude, potentially aiding grain growth. Furthermore, if solid particles have enough mass, and the particle size distribution extends to sufficiently large particle sizes, the solid component of the disc can undergo direct gravitational collapse to form bound clumps with masses typically between $1$ and $10$ M$_\oplus$. Thus, the concentration of dust in a self-gravitating disc could bypass the size barrier for collisional growth and directly form planetary cores early in the lifetime of the disc.         |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2505.00080/./SB_panel_v2.png', 'tmp_2505.00080/./J0100+2802_MUSE_1215.67_panel.png', 'tmp_2505.00080/./J0330-4025_MUSE_1215.67_panel.png', 'tmp_2505.00080/./J158-14_MUSE_1215.67_panel.png', 'tmp_2505.00080/./J0100+2802_channel_line_mean.png', 'tmp_2505.00080/./J0330-4025_channel_line_mean.png', 'tmp_2505.00080/./J158-14_channel_line_mean.png']
copying  tmp_2505.00080/./SB_panel_v2.png to _build/html/
copying  tmp_2505.00080/./J0100+2802_MUSE_1215.67_panel.png to _build/html/
copying  tmp_2505.00080/./J0330-4025_MUSE_1215.67_panel.png to _build/html/
copying  tmp_2505.00080/./J158-14_MUSE_1215.67_panel.png to _build/html/
copying  tmp_2505.00080/./J0100+2802_channel_line_mean.png to _build/html/
copying  tmp_2505.00080/./J0330-4025_channel_line_mean.png to _build/html/
copying  tmp_2505.00080/./J158-14_channel_line_mean.png to _build/html/
exported in  _build/html/2505.00080.md
    + _build/html/tmp_2505.00080/./SB_panel_v2.png
    + _build/html/tmp_2505.00080/./J010

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\msol}{M_\odot}$
$\newcommand{\vxhi}{\bar{x}_{\rm HI}}$
$\newcommand{\logtq}{\log{t_{\rm Q}}}$
$\newcommand{\zbar}{\langle z \rangle }$
$\newcommand{\chimp}{{\rm c Mpc}/h}$
$\newcommand{\lya}{Ly\alpha}$
$\newcommand{\lyb}{Ly\beta}$
$\newcommand{\lyc}{Ly\gamma}$
$\newcommand{\ciisf}{[C II]}$
$\newcommand{\cii}{[C{\small II}]}$
$\newcommand{\civ}{C{\small IV}}$
$\newcommand{\mgii}{Mg{\small II}}$
$\newcommand{\oiii}{[O{\small III}]}$
$\newcommand{\ha}{H\alpha}$
$\newcommand{\ddu}[1]{\texttt{\color{red}[DD: #1]}}$
$\newcommand{\change}[1]{\textbf{#1}}$</div>



<div id="title">

# Quasar lifetime measurements from extended $\lya$ nebulae at $z\sim 6$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.00080-b31b1b.svg)](https://arxiv.org/abs/2505.00080)<mark>Appeared on: 2025-05-02</mark> -  _14 pages, 7 figures_

</div>
<div id="authors">

D. {. D. c}{í}kov{á}, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** The existence of billion-solar-mass black holes hosted in luminous quasars within the first gigayear of cosmic history poses a challenge to our understanding of supermassive black hole (SMBH) growth. The problem is further exacerbated by the very short quasar lifetimes of $t_{\rm Q}\lesssim 10^6$ years, as derived from the extent of their proximity zone (PZ) sizes observed in the quasars' rest-UV spectra. However, the quasar lifetime estimates based on the extents of the proximity zones may be underestimated, as time-variable obscuration effects might have limited the quasars' emission along our sightline in the past.In this work, we present independent quasar lifetime measurements for six quasars at $z \sim 6$ leveraging the extended nebular emission _perpendicular_ to our line-of-sight. We use observations from the Very Large Telescope/Multi-Unit Spectroscopic Explorer (MUSE) to search for extended $\lya$ emission in the circumgalactic medium around quasars with small proximity zones and estimate their lifetimes as the light travel time between the SMBH and the outer edge of the nebula. We find agreement between the independent lifetime estimates. For one object we find a proximate absorption system prematurely truncating the extent of the quasar's proximity zone, which thus results in an expected discrepancy between the lifetime estimates. Our results provide further evidence that the quasars' current accretion episode has only recently begun, challenging our models of SMBH growth.

</div>

<div id="div_fig1">

<img src="tmp_2505.00080/./SB_panel_v2.png" alt="Fig4" width="100%"/>

**Figure 4. -** The extracted surface brightness profiles for the three quasars with nebular detections as a function of both the radial offset and the light travel time from the quasar. The black data points correspond to the surface brightness profile of the median nebula, as described in \S \ref{sec:nebulae}, where we only take into account the nebular pixels so as not to dilute signal at the edge of the nebula. We also show the surface brightness profile calculated via annular averaging of the full narrowband image of the detected nebula as gray data points to illustrate this signal dilution. In the background, we display the median stacked surface brightness profile of $z>5.7$ quasars from farina_mapping_2017 for comparison. The red dotted lines display the $16$th and $84$th percentile spread of the nebular extent when the nebular search parameters are varied. The quasar lifetime, $t_{\rm Q}^{\rm Ly\alpha}$, is measured as the light travel time from the quasar to the outer edge of the detected nebula in the transverse direction. Note that the horizontal error bars represent the binning of the annular apertures used to calculate the surface brightness. We also display the $1\sigma$(dark gray) and $2\sigma$(light gray) background derived from the sky variance in the outermost annular aperture. (*fig:SB*)

</div>
<div id="div_fig2">

<img src="tmp_2505.00080/./J0100+2802_MUSE_1215.67_panel.png" alt="Fig1.1" width="33%"/><img src="tmp_2505.00080/./J0330-4025_MUSE_1215.67_panel.png" alt="Fig1.2" width="33%"/><img src="tmp_2505.00080/./J158-14_MUSE_1215.67_panel.png" alt="Fig1.3" width="33%"/>

**Figure 1. -** Quasars in our sample with extended nebular emission. From left to right, the panels display pseudo-narrowband images of 1) the quasar's emission around $\lya$, 2) the PSF extracted from the broad wing of the quasar's $\lya$ line, 3) the PSF-subtracted data, 4) the $\chi$ data, essentially representing the SNR of the PSF-subtracted data, and 5) the smoothed $\chi$ data showing the extended nebular emission. Panels 1, 3, 4, and 5 are collapsed across the wavelength range of the detected nebula, and panel 2 is collapsed across the wavelength range of the PSF extraction as displayed in Appendix \ref{app:regions}. White patches correspond to masked foreground sources. (*fig:qsoswithnebulae*)

</div>
<div id="div_fig3">

<img src="tmp_2505.00080/./J0100+2802_channel_line_mean.png" alt="Fig3.1" width="33%"/><img src="tmp_2505.00080/./J0330-4025_channel_line_mean.png" alt="Fig3.2" width="33%"/><img src="tmp_2505.00080/./J158-14_channel_line_mean.png" alt="Fig3.3" width="33%"/>

**Figure 3. -** Spectral decomposition of the three nebular detections. The left three panels display the detected nebulae in three spectral channels of equal widths ($822 {\rm km/s}$ for J0330--4025, $946 {\rm km/s}$ for J158--14, and $1458 {\rm km/s}$ for J0100+2802), whereby the middle panel is centered on the quasar's systemic redshift as given by \cref{tab:sample}. We also color-code each displayed voxel by the velocity offset from the quasar's $\lya$ emission. The right panel shows the integrated emission line profile of the detected nebula. (*fig:velocity*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.00080"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\ay}[1]{\textcolor{red}{#1}}$
$\newcommand{\hans}[1]{\textcolor{cyan}{#1}}$
$\newcommand{\cass}[1]{\textcolor{teal}{\textbf{Cass:} #1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Dust density enhancements and the direct formation of planetary cores in gravitationally unstable discs

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.00363-b31b1b.svg)](https://arxiv.org/abs/2505.00363)<mark>Appeared on: 2025-05-02</mark> -  _15 pages, 11 figures, accepted for publication in Monthly Notices of the Royal Astronomical Society_

</div>
<div id="authors">

K. Rice, et al. -- incl., <mark>H. Baehr</mark>

</div>
<div id="abstract">

**Abstract:** Planet formation via core accretion involves the growth of solids that can accumulate to form planetary cores.  There are a number of barriers to the collisional growth of solids in protostellar discs, one of which is the drift, or metre, barrier.  Solid particles experience a drag force that will tend to cause them to drift towards the central star in smooth, laminar discs, potentially removing particles before they grow large enough to decouple from the disc gas. Here we present 3-dimensional, shearing box simulations that explore the dynamical evolution of solids in a protostellar disc that is massive enough for the gravitational instability to manifest as spiral density waves.  We expand on earlier work by considering a range of particle sizes and find that the spirals can still enhance the local solid density by more than an order of magnitude, potentially aiding grain growth.  Furthermore, if solid particles have enough mass, and the particle size distribution extends to sufficiently large particle sizes, the solid component of the disc can undergo direct gravitational collapse to form bound clumps with masses typically between $1$ and $10$ M $_\oplus$ . Thus, the concentration of dust in a self-gravitating disc could bypass the size barrier for collisional growth and directly form planetary cores early in the lifetime of the disc.

</div>

<div id="div_fig1">

<img src="tmp_2505.00363/./Figures/dustdens_multi_Fig6.png" alt="Fig9" width="100%"/>

**Figure 9. -** Surface density structure for all the particles in the simulation in which the particles have a mass $10^{-4}$(top left), $10^{-3}$(top right), $0.003$(middle left), $0.007$(middle right), $0.01$(bottom left) and $0.025$(bottom right) that of the gas. The particle self-gravity, and the backreaction on the gas, is included, and the particle size distribution covers 4 orders of magnitude in particle size with representative Stokes numbers ranging from $\mathrm{St} \sim 0.02$ to $\mathrm{St} \sim 200$. Apart from the $m_{\rm d} = 0.025 m_{\rm g}$ simulation, these are all at $t\Omega = 300$. For $m_{\rm d} = 0.025 m_{\rm g}$ the high densities of the clumps reduced the timestep and the simulation was stopped at $t \Omega = 220$. The red circles in the bottom two panels highlight dense clumps that have formed in these simulations. (*fig:ptclsfullrange*)

</div>
<div id="div_fig2">

<img src="tmp_2505.00363/./Figures/gasdens_slice_multi.png" alt="Fig1" width="100%"/>

**Figure 1. -** Gas density structure in the midplane (top panel) and a vertical slice through the disc at $x = 0$(bottom panel) at $t \Omega = 300$.  The disc has settled into a quasi-steady, self-gravitating state with spiral density waves and the simulation is clearly able to capture the vertical structure in the gas disc. (*fig:refgas*)

</div>
<div id="div_fig3">

<img src="tmp_2505.00363/./Figures/dustdens_multi_Fig2.png" alt="Fig2" width="100%"/>

**Figure 2. -** Surface density structure of the particle disc at $t \Omega = 300$, for the full size range of solid particles from the test particle simulation.  The top panel shows the particle surface density in the disc midplane, while the bottom panel shows the surface, or column, density projected on to the xz-plane. (*fig:refdust*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.00363"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

474  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

24  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
